## Data Processing - ECS171 Project Group 8

**Description**: Converts true/false entries to 1/0 and normalizes numerical data

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math

In [2]:
path_nontrend = 'datasets/11.09 nontrending.csv'
path_trend = 'datasets/11.09 trending.csv'
df_nontrend = pd.read_csv(path_nontrend)
df_trend = pd.read_csv(path_trend)
df_nontrend.head()

df_nontrend = df_nontrend.drop(['dimension'],axis=1)
df_trend = df_trend.drop(['dimension'],axis=1)
pd.set_option('display.max_columns', None)


/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (18,19,23) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
def handle_duplicates(dfn, dft):
    
    trend_later_list = []
    drop_list = []
    
    for row in dfn.index:
        if dfn['video_id'][row] in dft['video_id']:
            trend_later_list.append(dfn['video_id'][row])
            drop_list(row)
    
    dft['trend_later'] = False
    for ids in trend_later_list:
        dft.iloc[dft[dft['video_id'] == ids].index.values]['trend_later'] = True
    dfn.drop(drop_list, axis = 0, inplace = True)
    dfn['trend_later'] = False
    
def excess_remove(dft, dfn):
    excess = []
    for col in dfn.columns:
        if col not in dft.columns:
            excess.append(col)
    dfn.drop(excess, axis = 1, inplace = True)
    
def update_remove(df, kept_date):
    prospects = []
    remove = []
    for cols in df.columns:
        if "timestamp" in cols or "likes" in cols or "dislikes" in cols or "comment" in cols or "view" in cols:
            prospects.append(cols)
    for col2 in prospects:
        if kept_date not in col2 and  "Channel_viewCount" not in col2 :
            remove.append(col2)
    return remove

#convert quotations '\"'
#function: remove_quote(column)
#use on nontrend (definition, Channel_country) and trend (Channel_country)
def remove_quote(col):
    lst = []
    for v in col:
        if type(v) == str:
            lst.append(v.replace('"',''))
        else:
            lst.append(v)
    return lst


#sets 'tags' to number of tags
#function: count_tags(dataframe)
#use on nontrend and trend
def count_tags(df):
    lst=[]
    for entry in df['tags']:
        if entry == '[none]':
            lst.append(0)
        else:
            num_tags = entry.count('|')
            lst.append(num_tags)
    df['tags'] = lst


#Channel_country: 0 for INTL, 1 for USA
#function: encode_country(dataframe)
#use on nontrend and trend
def encode_country(df):
    lst = []
    for entry in df['Channel_country']:
        if entry == 'US':
            lst.append("USA")
        elif entry == "":
            lst.append("UNK")
        else:
            lst.append("INTL")
    df['Channel_country'] = lst

#remove Channel_hiddenSubscriberCount == True rows
#function: clean_subcount(dataframe)
#use on nontrend
def clean_subcount(df):
    lst = []
    for i in df.index:
        if df['Channel_hiddenSubscriberCount'][i] == True:
            lst.append(i)
    return lst

In [4]:
handle_duplicates(df_nontrend, df_trend)

In [5]:
from os import listdir
from os.path import isfile, join
import pandas as pd
#if a video is in both '11.09 nontrending' and '11.09 trending', drop it from nontrending (it was never nontrending.) If it shows up in one of the later trending video sets, add that information

#pass trending dframe as dft, nontrending as dfn
def handle_trending(dft,dfn):
    path= './datasets'
    trending = [f for f in listdir(path) if isfile(join(path, f)) and " trending" in f]

    all_trending_ids = set()
    for i in trending:
        all_trending_ids.update(pd.read_csv(f"{path}/{i}")["video_id"].to_list())

    original_trending_ids = dft["video_id"].to_list()

    trended = set()
    duplicate = set()
    for i in df.index:
        if df["video_id"][i] in all_trending_ids:
            trended.update([i])
        if dfn["video_id"][i] in original_trending_ids:
            duplicate.update([i])
    trended = trended.symmetric_difference(duplicate)

    #drop videos which are duplicates of videos in the "11.09 trending" database
    dfn.drop(duplicate, inplace=True)

    #these videos were not originally trending, but started trending later. 
    dfn.insert(10,"trended_later",False)
    dfn.loc[trended,"trended_later"]=True

In [6]:
df_trend['trending?'] = True
df_nontrend['trending?'] = False

In [7]:
excess_remove(df_trend, df_nontrend)
frames = [df_trend, df_nontrend]
# Link that teaches how to concatenate:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
df = pd.concat(frames, ignore_index=True, sort=False)
#df.reset_index(drop=True)
#df = df_trend
df = df.drop(['Unnamed: 0','is_trending'], axis=1)
#df.to_csv("~/CS/ECS-171-Group8/orig_DataProcessNotebook.csv")
#df.reset_index(drop=True)
df.dropna(inplace= True)

In [8]:
# df[df['duration'] != type(str)]

index = 0
lst_drop = []
for i in df['duration']:
    if type(i) != str:
        lst_drop.append(index)
    index += 1
df.drop(lst_drop, inplace = True)   

In [9]:
#-----------------------------
# installation: pip3 install isodate
import isodate
duration_seconds = []

for i in df['duration']:
    dur = isodate.parse_duration(i.replace('"','' ))
    duration_seconds.append(dur.total_seconds())
df['duration'] = duration_seconds

In [10]:
def engagementRate(likes, comment, subscriber):
    return (likes + comment)/subscriber

#drop any na values from the dataset
df = df.dropna()

like_col = [col for col in df if col.startswith('likes') ]
comment_col = [col for col in df if col.startswith('comment') ]
subscriber_col = [col for col in df if col.startswith('Channel_subscriberCount') ]

#feed the data to the formula and store the result in engagement rate
df["engagement_rate"] = engagementRate(df[like_col[len(like_col)-1]],df[comment_col[len(comment_col)-1]], df[subscriber_col[len(subscriber_col)-1]])

#replace any infinite values with nan
df["engagement_rate"] = df["engagement_rate"].replace([np.inf, -np.inf], np.nan)

In [11]:
new_df = df.drop(update_remove(df, "11_19"), axis = 1)

In [12]:
from sklearn.preprocessing import StandardScaler
numeric_cols = ['duration', 'Channel_viewCount', 'Channel_subscriberCount', 'Channel_videoCount', 'view_count_update_11_19_14', 'likes_update_11_19_14', 'dislikes_update_11_19_14', 'comment_count_update_11_19_14',]
new_df[numeric_cols] = StandardScaler().fit_transform(new_df[numeric_cols])
new_df.head()

,video_id,title,publishedAt,channelId,channelTitle,categoryId,duration,definition,time_retrieved,tags,thumbnail_link,ratings_disabled,description,Channel_viewCount,Channel_subscriberCount,Channel_hiddenSubscriberCount,Channel_videoCount,Channel_title,Channel_description,Channel_publishedAt,Channel_country,view_count_update_11_19_14,likes_update_11_19_14,dislikes_update_11_19_14,comment_count_update_11_19_14,11_19_14_update_timestamp,trend_later,trending?,engagement_rate
0,vJYL4Osyipc,Biden Victory Cold Open - SNL,2020-11-08T06:34:11Z,UCqFzWxSCi39LnW1JKFR3efg,Saturday Night Live,24.0,-0.070631,sd,2020-11-09T00:54:13Z,snl|saturday night live|season 46|snl 46|snl c...,https://i.ytimg.com/vi/vJYL4Osyipc/default.jpg,False,Joe Biden (Jim Carrey) and Donald Trump (Alec ...,0.821677,0.504660,False,-0.189337,"""Saturday Night Live""","""Welcome to the official Saturday Night Live c...","""2013-07-23T21:32:27Z""","""US""",6.070299,3.317911,10.219838,2.831197,2020-11-19T14:21:04Z,False,True,0.028492
1,moOxq_8l_34,How President Trump and the White House reacte...,2020-11-08T00:34:15Z,UCrp_UI8XtuYfpiqluWLD7Lw,CNBC Television,25.0,-0.085616,hd,2020-11-09T00:54:13Z,news with shepard smith|shepard smith|news|eve...,https://i.ytimg.com/vi/moOxq_8l_34/default.jpg,False,CNBC's Kayla Tausche joins The News with Shepa...,-0.275024,-0.351021,False,1.158788,"""CNBC Television""","""""","""2018-05-29T15:09:14Z""","""US""",2.959261,0.560667,4.950745,3.116102,2020-11-19T14:21:04Z,False,True,0.090361
2,452iQ8ttIcc,Watch Live: President-Elect Joe Biden Addresse...,2020-11-08T02:08:14Z,UCBi2mrWuNuyYy4gbM6fU18Q,ABC News,25.0,0.095291,hd,2020-11-09T00:54:13Z,2020 election|2020 presidential election|trump...,https://i.ytimg.com/vi/452iQ8ttIcc/default.jpg,False,#Election2020 #Election #JoeBiden #PresidentEl...,0.651650,0.461225,False,1.270032,"""ABC News""","""Make ABC News your daily news outlet for brea...","""2006-08-07T23:12:21Z""","""US""",1.346030,0.520324,2.289273,1.055055,2020-11-19T14:21:04Z,False,True,0.006721
3,KW4pQFHKziI,I GOT PLASTIC SURGERY TO SEE HOW MY FRIENDS RE...,2020-11-07T21:30:34Z,UCPpATKqmMV-CNRNWYaDUwiA,Alexa Rivera,26.0,-0.070955,hd,2020-11-09T00:54:13Z,[none],https://i.ytimg.com/vi/KW4pQFHKziI/default.jpg,False,"This was definitely such a fun prank, their re...",-0.258244,0.002545,False,-0.407834,"""Alexa Rivera""","""""","""2010-08-21T04:33:50Z""","""""",1.853380,2.596665,2.745624,0.712809,2020-11-19T14:21:04Z,False,True,0.045112
4,uyZ8GwksYqk,FC Barcelona 5 - 2 Real Betis - HIGHLIGHTS & G...,2020-11-07T17:31:34Z,UC0YatYmg5JRYzXJPxIdRd8g,beIN SPORTS USA,17.0,-0.072795,hd,2020-11-09T00:54:13Z,beIN SPORTS|beIN SPORTS USA|Futbol|Football|la...,https://i.ytimg.com/vi/uyZ8GwksYqk/default.jpg,False,Lionel Messi entró al segundo tiempo y resucit...,-0.330856,-0.408529,False,-0.204068,"""beIN SPORTS USA""","""The fastest growing network in the US offerin...","""2012-07-16T20:11:38Z""","""US""",0.272154,-0.076487,0.044110,0.018855,2020-11-19T14:21:04Z,False,True,0.029918


In [13]:
# #plt.hist(new_df['Channel_country'])
# plt.subplots(figsize=(71,71))
# sns.countplot(new_df['Channel_country'])

In [14]:
new_df['definition'] = remove_quote(new_df['definition'])
new_df['Channel_country'] = remove_quote(new_df['Channel_country'])

count_tags(new_df)

encode_country(new_df)
new_df[pd.get_dummies(new_df['Channel_country']).columns] = pd.get_dummies(new_df['Channel_country'])[:]

new_df = new_df.drop(clean_subcount(new_df))


In [15]:
new_df

,video_id,title,publishedAt,channelId,channelTitle,categoryId,duration,definition,time_retrieved,tags,thumbnail_link,ratings_disabled,description,Channel_viewCount,Channel_subscriberCount,Channel_hiddenSubscriberCount,Channel_videoCount,Channel_title,Channel_description,Channel_publishedAt,Channel_country,view_count_update_11_19_14,likes_update_11_19_14,dislikes_update_11_19_14,comment_count_update_11_19_14,11_19_14_update_timestamp,trend_later,trending?,engagement_rate,INTL,UNK,USA
0,vJYL4Osyipc,Biden Victory Cold Open - SNL,2020-11-08T06:34:11Z,UCqFzWxSCi39LnW1JKFR3efg,Saturday Night Live,24.0,-0.070631,sd,2020-11-09T00:54:13Z,36,https://i.ytimg.com/vi/vJYL4Osyipc/default.jpg,False,Joe Biden (Jim Carrey) and Donald Trump (Alec ...,0.821677,0.504660,False,-0.189337,"""Saturday Night Live""","""Welcome to the official Saturday Night Live c...","""2013-07-23T21:32:27Z""",USA,6.070299,3.317911,10.219838,2.831197,2020-11-19T14:21:04Z,False,True,0.028492,0,0,1
1,moOxq_8l_34,How President Trump and the White House reacte...,2020-11-08T00:34:15Z,UCrp_UI8XtuYfpiqluWLD7Lw,CNBC Television,25.0,-0.085616,hd,2020-11-09T00:54:13Z,28,https://i.ytimg.com/vi/moOxq_8l_34/default.jpg,False,CNBC's Kayla Tausche joins The News with Shepa...,-0.275024,-0.351021,False,1.158788,"""CNBC Television""","""""","""2018-05-29T15:09:14Z""",USA,2.959261,0.560667,4.950745,3.116102,2020-11-19T14:21:04Z,False,True,0.090361,0,0,1
2,452iQ8ttIcc,Watch Live: President-Elect Joe Biden Addresse...,2020-11-08T02:08:14Z,UCBi2mrWuNuyYy4gbM6fU18Q,ABC News,25.0,0.095291,hd,2020-11-09T00:54:13Z,13,https://i.ytimg.com/vi/452iQ8ttIcc/default.jpg,False,#Election2020 #Election #JoeBiden #PresidentEl...,0.651650,0.461225,False,1.270032,"""ABC News""","""Make ABC News your daily news outlet for brea...","""2006-08-07T23:12:21Z""",USA,1.346030,0.520324,2.289273,1.055055,2020-11-19T14:21:04Z,False,True,0.006721,0,0,1
3,KW4pQFHKziI,I GOT PLASTIC SURGERY TO SEE HOW MY FRIENDS RE...,2020-11-07T21:30:34Z,UCPpATKqmMV-CNRNWYaDUwiA,Alexa Rivera,26.0,-0.070955,hd,2020-11-09T00:54:13Z,0,https://i.ytimg.com/vi/KW4pQFHKziI/default.jpg,False,"This was definitely such a fun prank, their re...",-0.258244,0.002545,False,-0.407834,"""Alexa Rivera""","""""","""2010-08-21T04:33:50Z""",UNK,1.853380,2.596665,2.745624,0.712809,2020-11-19T14:21:04Z,False,True,0.045112,0,1,0
4,uyZ8GwksYqk,FC Barcelona 5 - 2 Real Betis - HIGHLIGHTS & G...,2020-11-07T17:31:34Z,UC0YatYmg5JRYzXJPxIdRd8g,beIN SPORTS USA,17.0,-0.072795,hd,2020-11-09T00:54:13Z,12,https://i.ytimg.com/vi/uyZ8GwksYqk/default.jpg,False,Lionel Messi entró al segundo tiempo y resucit...,-0.330856,-0.408529,False,-0.204068,"""beIN SPORTS USA""","""The fastest growing network in the US offerin...","""2012-07-16T20:11:38Z""",USA,0.272154,-0.076487,0.044110,0.018855,2020-11-19T14:21:04Z,False,True,0.029918,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22729,vDkRDSRb-3o,13 Year Old Bunchie Young Had A CRAZY SUMMER! ...,2020-11-08T17:03:45Z,UCDtwmlSRVWsZsQSJjLS78VA,Overtime SZN,17.0,-0.061813,hd,2020-11-09T00:58:05Z,20,https://i.ytimg.com/vi/vDkRDSRb-3o/default.jpg,False,Bunchie Episode 1 ▻▻ https://ovrt.me/3nkLxie M...,-0.329715,-0.406792,False,-0.393614,"""Overtime SZN""","""Based in NYC, Overtime is a sports network fo...","""2017-08-24T21:14:21Z""",UNK,-0.211970,-0.187079,-0.201671,-0.121375,2020-11-19T14:21:05Z,False,False,0.003319,0,1,0
22730,WjFbwUpqTr0,What if &#39;Among Us&#39; Happened in Real Life?,2020-11-08T17:03:34Z,UCphTF9wHwhCt-BzIq-s4V-g,What If,28.0,-0.075878,hd,2020-11-09T00:58:05Z,27,https://i.ytimg.com/vi/WjFbwUpqTr0/default.jpg,False,Get more insightful information about the game...,-0.286089,-0.056527,False,-0.397274,"""What If""","""An epic exploration of possibilities. What If...","""2018-07-18T15:31:35Z""",INTL,-0.147779,-0.058773,-0.043916,-0.010100,2020-11-19T14:21:05Z,False,False,0.002862,1,0,0
22731,mlvdrLuvAUs,[LAUGHS IN 71 ROUND DI

# Sources
1. https://stackoverflow.com/questions/40950791/remove-quotes-from-string-in-python
